# Interactive graph

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/drug_interactions/database/preprocessed_drug_interactions_db.csv', index_col=0)
df

,query_drug_link,interaction_drug_link,drug_names,interaction_drug_names
0,https://www.drugs.com/cons/a-b-otic.html,NaN,a-b-otic,[]
1,https://www.drugs.com/mtm/abacavir.html,"https://www.drugs.com/mtm/riociguat.html, http...",abacavir,"['riociguat', 'methadone']"
2,https://www.drugs.com/mtm/abacavir-and-lamivud...,NaN,abacavir-and-lamivudine,[]
3,https://www.drugs.com/pro/abacavir-and-lamivud...,NaN,abacavir-and-lamivudine-tablets,[]
4,https://www.drugs.com/pro/abacavir-oral-soluti...,NaN,abacavir-oral-solution,[]
...,...,...,...,...
89248,https://www.drugs.com/pro/zyvana.html,NaN,zyvana,[]
89249,https://www.drugs.com/zyvox.html,NaN,zyvox,[]
89250,https://www.drugs.com/cons/zyvox-linezolid-int...,NaN,zyvox-linezolid-intravenous,[]
89251,https://www.drugs.com/cons/zyvox-linezolid-ora...,NaN,zyvox-linezolid-oral,[]


In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from networkx.algorithms.community import greedy_modularity_communities
import re

In [ ]:
df['interaction_drug_names'] = df['interaction_drug_names'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [ ]:

# Create a graph
G = nx.Graph()

# Add nodes and edges
for index, row in df.iterrows():
    drug = row['drug_names'].strip()  # Ensure no leading/trailing spaces
    interactions = row['interaction_drug_names']
    if isinstance(interactions, list) and interactions:  # Check if interactions is a list and not empty
        for interaction in interactions:
            G.add_edge(drug, interaction.strip())  # Ensure no leading/trailing spaces

# Network analysis
centrality = nx.degree_centrality(G)
nx.set_node_attributes(G, centrality, 'centrality')

In [ ]:
# Function to get the subgraph centered around a specified drug
def get_drug_subgraph(drug_name, G, radius=1):
    drug_name = drug_name.strip()  # Ensure no leading/trailing spaces
    print(f"Searching for drug: '{drug_name}'")  # Debug statement
    if drug_name in G:
        nodes = [drug_name] + list(nx.single_source_shortest_path_length(G, drug_name, radius).keys())
        print(f"Nodes in subgraph: {nodes}")  # Debug statement
        return G.subgraph(nodes)
    else:
        print(f"Drug '{drug_name}' not found in the network.")  # Debug statement
        return None

In [ ]:
# Function to visualize the subgraph using Plotly
def visualize_subgraph(G_sub, drug_name):
    pos = nx.spring_layout(G_sub)
    edge_x = []
    edge_y = []
    for edge in G_sub.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_color = []
    node_size = []
    for node in G_sub.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_color.append(G_sub.nodes[node]['centrality'])
        node_size.append(G_sub.nodes[node]['centrality'] * 100)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=[node for node in G_sub.nodes()],
        textposition="top center",
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            size=node_size,
            color=node_color,
            colorbar=dict(
                thickness=15,
                title='Node Centrality',
                xanchor='left',
                titleside='right'
            )
        )
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=f'Drug Interaction Network centered on {drug_name}',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()


In [ ]:
    #abacavir

    drug_name = input("Enter the name of the drug: ")
    G_sub = get_drug_subgraph(drug_name, G)
    print(G_sub)
    if G_sub:
        visualize_subgraph(G_sub, drug_name)
    else:
        print("Drug not found in the network.")